In [1]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr
pd.options.display.float_format = "{:.4f}".format
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (16,10)
# plt.style.use("~/.dracula.mplstyle") 
import requests
import quandl
import functools
import re
import statsmodels.api as sm

# QUANDL Config
import os, sys
parent_path = os.path.abspath(os.path.join('..'))
if parent_path not in sys.path:
    sys.path.append(parent_path)
from common.config import QUANDL_API_KEY  # If you are not Philip, you don't have config
                                          # unless you use your own API key

quandl.ApiConfig.api_key = QUANDL_API_KEY # Replace with your API key if running.

In [2]:
START_DATE    = "2019-12-02"
END_DATE      = "2021-12-31"

QUANDL_DB     = "QUOTEMEDIA/PRICES"
QUANDL_ASSETS = ("FCOM", "VOX")

FF_TABLE      = "F-F_Research_Data_Factors_daily"
FF_DB         = "famafrench"

In [3]:
if "pull_quandl" in globals():
    print("Not re-defining.")
else:
    print("Defining quandl pull")
    @functools.lru_cache(maxsize=16)
    def pull_quandl(db, assets, start_date, end_date):
        ret = quandl.get_table(db, ticker=list(assets), date={"gte":START_DATE, "lte":END_DATE})
        return ret

eod_raw = pull_quandl(QUANDL_DB, QUANDL_ASSETS, START_DATE, END_DATE)
display(eod_raw)

Defining quandl pull


,ticker,date,open,high,low,close,volume,dividend,split,adj_open,adj_high,adj_low,adj_close,adj_volume
None,,,,,,,,,,,,,,
0,VOX,2021-12-31,136.8700,137.3300,135.5100,135.5600,78065.0000,0.0000,1.0000,136.8700,137.3300,135.5100,135.5600,78065.0000
1,VOX,2021-12-30,136.7800,137.9900,136.7800,137.1900,114214.0000,0.0000,1.0000,136.7800,137.9900,136.7800,137.1900,114214.0000
2,VOX,2021-12-29,137.1000,137.5300,136.0950,136.5000,97534.0000,0.0000,1.0000,137.1000,137.5300,136.0950,136.5000,97534.0000
3,VOX,2021-12-28,137.4800,138.4700,137.0400,137.2300,119793.0000,0.0000,1.0000,137.4800,138.4700,137.0400,137.2300,119793.0000
4,VOX,2021-12-27,136.8100,137.7915,136.5800,137.4700,94772.0000,0.0000,1.0000,136.8100,137.7915,136.5800,137.4700,94772.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1047,FCOM,2019-12-06,35.2800,35.2800,35.1500,35.2100,43170.0000,0.0000,1.0000,34.6977,34.6977,34.5698,34.6288,43170.0000
1048,FCOM,2019-12-05,35.0800,35.0800,34.8100,34.9900,54791.0000,0.0000,1.0000,34.5010,34.5010,34.2354,34.4125,54791.0000
1049,FCOM,2019-12-04,34.8300,35.0000,34.8000,34.8500,59115.0000,0.0000,1.0000,34.2551,34.4223,34.2256,34.2748,59115.0000


In [4]:
ff_raw = pdr.data.DataReader(FF_TABLE, FF_DB, 
                            ''.join(START_DATE.split('-')),
                            ''.join(END_DATE.split('-')))

ff_raw[0]

,Mkt-RF,SMB,HML,RF
Date,,,,
2019-12-02,-0.8700,-0.2400,0.4600,0.0070
2019-12-03,-0.6600,0.6000,-0.8300,0.0070
2019-12-04,0.6000,0.1300,0.2500,0.0070
2019-12-05,0.1300,-0.2100,0.4600,0.0070
2019-12-06,0.9100,0.2400,0.3900,0.0070
...,...,...,...,...
2021-11-23,-0.0300,-0.5100,1.7400,0.0000
2021-11-24,0.3000,0.0200,-0.8600,0.0000
2021-11-26,-2.2200,-1.2300,-0.9300,0.0000
